# -1) imports

In [1]:
import socket
import random
import time

In [2]:
from datetime import datetime

In [3]:
def get_time():
    from datetime import datetime
    _ = datetime.now()
    return _

def print_time():
    print(get_time())

In [4]:
print_time()

2019-04-27 15:29:42.164287


# 0) configuration

In [5]:
with open("port.param", "r") as f:
    port           = int(f.readline()) + 1

In [6]:

ip   = socket.gethostname()

In [7]:
with open("port.param", "w") as f:
    f.write(str(port))


# 1) création d'une socket

In [8]:
serversocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
serversocket.bind((ip, port))
print_time()
print("listen on %s:%s"%(ip, port))
serversocket.listen(1)

2019-04-27 15:29:42.248049
listen on 337b0ed4fa15:12039


# 2) attendre que spark se connecte

In [9]:
(clientsocket, address) = serversocket.accept()

In [10]:
print("Connection de %s :\n %s"%(address, clientsocket))

Connection de ('172.17.0.2', 48934) :
 <socket.socket fd=45, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('172.17.0.2', 12039), raddr=('172.17.0.2', 48934)>


# 3) envoie de données

In [11]:
serversocket.setblocking(False) 

In [12]:
nb_client = 3
nb_achat  = 5
clients   = ["client_%s"%x for x in range(nb_client)]
entreprises = ["LVMH"]#, "L'OREAL", "TOTAL", "AIRBUS GROUP (EX-EADS)", "SANOFI", "VALEO"]
entreprises = ["LVMH", "L'OREAL", "TOTAL", "AIRBUS GROUP (EX-EADS)", "SANOFI", "VALEO"]
entreprises = ["LVMH", "L'OREAL"]#, "TOTAL", "AIRBUS GROUP (EX-EADS)", "SANOFI", "VALEO"]


In [13]:
from collections import defaultdict

In [14]:
tps_attente      = 1
liste_des_envois = []
# on va effectuer 4 envois
for i in range(4):
    time.sleep(tps_attente)
    # on envoie la valeur 100 pour la première entreprise, et 200 pour la seconde
    achats      = [(entreprise,100*(i+1)) for i,entreprise in enumerate(entreprises)]
    # on fait deux envois par seconde, afin de vérifier la capacité de Spark
    # de cumuler les informations par secondes
    for i in range(2):
        for client, valeur in achats:
            # on envoie les informations en forme de chaine de caractère
            to_send = '%s,%s\n'%(client, valeur)                
            clientsocket.send(to_send.encode())
            # on enregistre les événements pour pouvoir vérifier ensuite
            # les résultats via python et pandas
            dico         = {client : valeur }
            dico["time"] = get_time()
            liste_des_envois.append(dico)

2019-04-27 15:30:15.548120
sent :  LVMH,100

sent 9 bytes
2019-04-27 15:30:15.553198
sent :  L'OREAL,200

sent 12 bytes
2019-04-27 15:30:15.553568
sent :  LVMH,100

sent 9 bytes
2019-04-27 15:30:15.553703
sent :  L'OREAL,200

sent 12 bytes
2019-04-27 15:30:16.555525
sent :  LVMH,100

sent 9 bytes
2019-04-27 15:30:16.556070
sent :  L'OREAL,200

sent 12 bytes
2019-04-27 15:30:16.557797
sent :  LVMH,100

sent 9 bytes
2019-04-27 15:30:16.558122
sent :  L'OREAL,200

sent 12 bytes
2019-04-27 15:30:17.559651
sent :  LVMH,100

sent 9 bytes
2019-04-27 15:30:17.560452
sent :  L'OREAL,200

sent 12 bytes
2019-04-27 15:30:17.560587
sent :  LVMH,100

sent 9 bytes
2019-04-27 15:30:17.561085
sent :  L'OREAL,200

sent 12 bytes
2019-04-27 15:30:18.562949
sent :  LVMH,100

sent 9 bytes
2019-04-27 15:30:18.563776
sent :  L'OREAL,200

sent 12 bytes
2019-04-27 15:30:18.563927
sent :  LVMH,100

sent 9 bytes
2019-04-27 15:30:18.564257
sent :  L'OREAL,200

sent 12 bytes


In [16]:
import pandas as pd
df = pd.DataFrame(r2).groupby("time").sum()

df["L'OREAL bis"] = df["L'OREAL"]
df["L'OREAL ter"] = df["L'OREAL"]
df["L'OREAL 4"] = df["L'OREAL"]
df["L'OREAL 5"] = df["L'OREAL"]
df["L'OREAL 6"] = df["L'OREAL"]

In [18]:
df[["L'OREAL", "L'OREAL bis", "L'OREAL ter", "L'OREAL 4", "L'OREAL 5", "L'OREAL 6"]]

,L'OREAL,L'OREAL bis,L'OREAL ter,L'OREAL 4,L'OREAL 5,L'OREAL 6
time,,,,,,
2019-04-27 15:30:15.553157,0.0,0.0,0.0,0.0,0.0,0.0
2019-04-27 15:30:15.553553,200.0,200.0,200.0,200.0,200.0,200.0
2019-04-27 15:30:15.553674,0.0,0.0,0.0,0.0,0.0,0.0
2019-04-27 15:30:15.553924,200.0,200.0,200.0,200.0,200.0,200.0
2019-04-27 15:30:16.556021,0.0,0.0,0.0,0.0,0.0,0.0
2019-04-27 15:30:16.557752,200.0,200.0,200.0,200.0,200.0,200.0
2019-04-27 15:30:16.558074,0.0,0.0,0.0,0.0,0.0,0.0
2019-04-27 15:30:16.558403,200.0,200.0,200.0,200.0,200.0,200.0
2019-04-27 15:30:17.560429,0.0,0.0,0.0,0.0,0.0,0.0
